**Title**: Illustrate reading CSV files within Dataviews in the SDK

**Date**:  February 16th 2023

**Description**:  
This notebook shows the approach for loading CSV files into a dataview.

See the online SDK help for more details: https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/data_views.html#files

# Requirements:
- Flywheel v16.8 or higher
- Flywheel SDK 16.15.9 or higher


## Install SDK and start client

In [ ]:
!pip install flywheel-sdk

In [ ]:
import pandas as pd

In [ ]:
import flywheel
from getpass import getpass
try:
    with open("api_key.txt") as file:
      api_key = file.read()
except FileNotFoundError:
    api_key = getpass("Enter your Flywheel API Key:")
    with open("api_key.txt",'w') as file:
      file.write(api_key)

In [ ]:
fw = flywheel.Client(api_key)
self = fw.get_current_user()
config = fw.get_config()
print('I am %s %s, connected to %s.' % (self.firstname, self.lastname, config['site']['api_url']))
version = fw.get_version()
print('Flywheel release: %s\nCLI version: %s\nDatabase version: %s' % (version.flywheel_release, version.cli_version, version.database))

print("API Docs:", config['site']['api_url']+"/docs")

# Set up the Example Project


In [ ]:
GROUP_ID = 'fmm'
PROJECT_LABEL = 'Test Dataview CSV File Reads'

In [ ]:
my_group = fw.lookup(GROUP_ID)


project = my_group.projects.find_first(f'label={PROJECT_LABEL}')
if not project: 
    project=my_group.add_project(label=PROJECT_LABEL)
    print("Project Created")
else:
  print("That project is already taken")

Project Created


### Add some files at the subject level
We are going to use some csv files that is provided within the colab notebook under the 'sample data' directory - 'california_housing_train.csv'

In [ ]:
df = pd.read_csv('/content/sample_data/california_housing_train.csv')
df.columns=['file_X1', 'file_X2', 'file_X3', 'file_X4', 'file_X5', 'file_X6', 'file_X7', 'file_X8', 'file_X9']
df.to_csv('data_file.csv')

In [ ]:
#Create 10 subjects and put a file under each
for s in range(0,10):
  subject_dict ={}
  subject_dict['label'] = "Subject" + str(s)
  subject = project.add_subject(subject_dict)
  subject = subject.reload()
  subject.upload_file('data_file.csv')

# Create and Run the Data View

### What columns do we want from the csv file?

Requires consistency in file and column naming


In [ ]:
csv_columns = ['file_X1', 'file_X3', 'file_X5', 'file_X7', 'file_X9']

## Create the dataview

In [ ]:
#pick the columns in the metadata
columns = ['subject.label',
           'file.file_id',
           'file.name' ] 

In [ ]:
# Specify the dataview
builder = flywheel.ViewBuilder(label='SDK CSV Aggregation',
                              columns = columns,
                              container='subject', #Needed for file metadata
                              filename='data_file.csv', # Needed for file metadata
                              match='newest',
                              process_files=True,
                              include_ids=False,
                              include_labels=False,
                              sort=False,
                              )


In [ ]:
#Use the newest file if there are dups in the match
#builder.file_match('newest')



In [ ]:
#Add our CSV columns to the builder
for c in csv_columns:
  builder.file_column(c, type='float')


In [ ]:
# Create the dataview specification
sdk_dataview = builder.build()

In [ ]:
# Create the Dataview in Flywheel
view_id = fw.add_view(project.id, sdk_dataview)

In [ ]:
#Create the dataview and execute it, waiting for return
#This may take a couple minutes
df = fw.read_view_dataframe(sdk_dataview, project.id)

In [ ]:
df

,subject.label,file.file_id,file.name,file_X1,file_X3,file_X5,file_X7,file_X9,errors
0,Subject0,63e423a417c0b262ffe542be,data_file.csv,-114.31,15,1283,472,66900,
1,Subject0,63e423a417c0b262ffe542be,data_file.csv,-114.47,19,1901,463,80100,
2,Subject0,63e423a417c0b262ffe542be,data_file.csv,-114.56,17,174,117,85700,
3,Subject0,63e423a417c0b262ffe542be,data_file.csv,-114.57,14,337,226,73400,
4,Subject0,63e423a417c0b262ffe542be,data_file.csv,-114.57,20,326,262,65500,
...,...,...,...,...,...,...,...,...,...
169995,Subject9,63e423ac17c0b262ffe542c7,data_file.csv,-124.26,52,394,369,111400,
169996,Subject9,63e423ac17c0b262ffe542c7,data_file.csv,-124.27,36,528,465,79000,
169997,Subject9,63e423ac17c0b262ffe542c7,data_file.csv,-124.30,17,531,456,103600,
169998,Subject9,63e423ac17c0b262ffe542c7,data_file.csv,-124.30,19,552,478,85800,


# Clean up the Flywheel Example Project

### Delete the Dataview

In [ ]:
for dv in fw.get_views(project.id):
  print(dv.label,end='---> ')
  if dv.label in ['SDK CSV Aggregation']:
    fw.delete_view(dv.id)
    print('deleted')
  else:
    print('remaining')

Files stats---> remaining
File Stats 2---> remaining
THIS IS A UX ISSUE---> remaining
13605---> remaining
SDK CSV Aggregation---> deleted


### Delete the subjects and the project

In [ ]:
# Clean up subjects
for s in range(0,10):

  subject_label = "Subject" + str(s)
  subject = project.subjects.find_first(f'label={subject_label}')
  fw.delete_subject(subject['_id'])

In [ ]:
#Clean up project
fw.delete_project(project['_id'])

{'deleted': 1}